In [22]:
import geopandas as gpd
import pandas as pd
import os
from shapely import wkt

# Group files by radio type
radio_files = {'2G': [], '3G': [], '4G': [], '5G': []}

# Find all relevant files and group them
for f in os.listdir():
    if f.endswith(('.gpkg', '.geojson')):
        for rt in ['2G', '3G', '4G', '5G']:
            if rt in f:
                radio_files[rt].append(f)

# Check for multiple files per radio type
for rt, files in radio_files.items():
    if len(files) > 1:
        print(f"WARNING: Multiple files found for {rt}: {files}")

# Process one file per radio type
concatenated = []
for rt, files in radio_files.items():
    if files:
        try:
            # Use the first file found for this radio type
            file_name = files[0]
            data = gpd.read_file(file_name)
            data['type'] = rt
            concatenated.append(data)
            print(f"Using {file_name} for {rt}")
        except Exception as e:
            print(f"Error reading {file_name}: {e}")

# Combine data
if concatenated:
    coverage_data = gpd.GeoDataFrame(pd.concat(concatenated, ignore_index=True))
    coverage_data["coverage"] = 1
    coverage_data['WKT'] = coverage_data['geometry'].apply(lambda geom: wkt.dumps(geom))
    coverage_data['crs'] = 'epsg:4326' 
    coverage_data.drop(columns=['geometry']).to_csv("coverage_all.csv")
else:
    print("No files processed")

Using coverage_2G.gpkg for 2G
Using coverage_3G.gpkg for 3G
Using coverage_4G.gpkg for 4G


In [15]:
coverage_data

,DN,coverage,geometry,type
0,1,1,"POLYGON ((33.72623 -9.54642, 33.72623 -9.55111...",2G
1,1,1,"POLYGON ((33.74049 -9.54877, 33.74049 -9.55111...",2G
2,1,1,"POLYGON ((33.73098 -9.54408, 33.73098 -9.54642...",2G
3,2,1,"POLYGON ((33.75 -9.55346, 33.75 -9.55814, 33.7...",2G
4,1,1,"POLYGON ((33.73336 -9.5558, 33.73336 -9.55814,...",2G
...,...,...,...,...
25451,1,1,"POLYGON ((35.17853 -16.13336, 35.18091 -16.133...",4G
25452,1,1,"POLYGON ((35.16665 -16.14478, 35.16665 -16.147...",4G
25453,1,1,"POLYGON ((35.17378 -16.1425, 35.17615 -16.1425...",4G
25454,1,1,"POLYGON ((34.99788 -15.55256, 35.00502 -15.552...",4G


In [2]:
pip install pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 111.0 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd

df = pd.read_parquet('MWI-mobilecoverage(1).parquet')
print(df.head())

   coverage processed_date
0         3     2025-02-18
1         3     2025-02-18
2         3     2025-02-18
3         3     2025-02-18
4         3     2025-02-18


In [4]:
df.columns

Index(['coverage', 'processed_date'], dtype='object')

In [6]:
df.coverage.value_counts()

coverage
3    21793587
1     1223452
2       51633
Name: count, dtype: int64